In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train = pd.read_csv('data/train_ver02.csv', index_col='STU_ID')
test = pd.read_csv('data/test_ver02.csv', index_col='STU_ID')
idx2job = pd.read_csv('data/idx2job.csv', index_col=0)
idx2job = idx2job['job'].to_dict()
train.head()

,X1SEX,X1RACE,X1HISPANIC,X1WHITE,X1BLACK,X1DUALLANG,X1STDOB,X1TXMTH,X1TXMSEM,X1TXMSCR,...,X1LOCALE,X1REGION,X1SCHOOLCLI,X1COUPERTEA,X1COUPERCOU,X1COUPERPRI,X1AQSTAT,X1STU30OCC2,birthyear,birthmonth
STU_ID,,,,,,,,,,,,,,,,,,,,,
10002,2,8,0,1.0,0.0,1.0,199511.0,-0.2956,0.2696,35.8045,...,4,1,0.32,-0.80,1.15,-1.61,1,0,5,11
10003,2,3,0,0.0,1.0,1.0,199506.0,1.2997,0.2555,56.0477,...,2,4,-0.61,0.81,-0.15,0.00,1,5,5,6
10004,2,8,0,1.0,0.0,1.0,199505.0,-0.1427,0.2649,38.0888,...,2,3,-2.14,0.03,0.26,0.20,1,0,5,5
10005,1,8,0,1.0,0.0,1.0,199505.0,1.1405,0.2312,54.1762,...,1,3,-0.65,-0.25,0.14,0.05,1,10,5,5
10006,2,8,0,1.0,0.0,1.0,199504.0,0.7106,0.2567,49.0531,...,4,3,-0.31,0.51,-0.90,0.69,1,0,5,4


In [ ]:
y = train.X1STU30OCC2
x = train.drop(columns='X1STU30OCC2')

In [ ]:
forest = RandomForestClassifier(n_estimators=300, n_jobs=-1)
forest.fit(x,y)

RandomForestClassifier(n_estimators=300, n_jobs=-1)

In [ ]:
# 0.37403
pred_base = pd.DataFrame({'STU_ID': test.index, 'X1STU30OCC2' : forest.predict(test)})
pred_base['X1STU30OCC2'] = pred_base['X1STU30OCC2'].replace(idx2job)
pred_base.to_csv('jobrec_baseline.csv', index=False, encoding='utf-8-sig')

In [ ]:
imp = forest.feature_importances_
idx = np.argsort(imp)[::-1]
labels = x.columns
pd.DataFrame({'columns':x.columns, 'imp':forest.feature_importances_}).sort_values(by='imp', ascending=False)

,columns,imp
66,X1STUEDEXPCT,0.023531
0,X1SEX,0.022778
8,X1TXMSEM,0.022386
54,X1SES_U,0.020889
65,X1SCHOOLENG,0.020816
...,...,...
71,X1PQSTAT,0.001097
17,X1MACC,0.000923
69,X1TESTSTAT,0.000887
70,X1SQSTAT,0.000664


In [ ]:
x_filter = x[labels[imp>0.01]] #threshold를 줘서 피쳐 줄였음
x_filter.shape
test_filter = test[x_filter.columns]

In [ ]:
forest_small = RandomForestClassifier(n_estimators=300, n_jobs=-1)
forest_small.fit(x_filter,y)

RandomForestClassifier(n_estimators=300, n_jobs=-1)

In [ ]:
#0.37959
g_small = pd.DataFrame({'STU_ID': test.index, 'X1STU30OCC2' : forest_small.predict(test_filter)})
g_small['X1STU30OCC2'] = g_small['X1STU30OCC2'].replace(idx2job)
g_small.to_csv('jobrec_small.csv', index=False, encoding='utf-8-sig')

In [ ]:
train_new = x_filter
train_new['X1STU30OCC2'] = y.values
train_new.reset_index().to_csv('data/train_ver03.csv',index=False, encoding='utf-8-sig')
test_filter.reset_index().to_csv('data/test_ver03.csv',index=False, encoding='utf-8-sig')

/var/folders/vt/76d2nkdd2msb93lkngc6yv1r0000gn/T/ipykernel_3221/692845606.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_new['X1STU30OCC2'] = y.values
